In [1]:
import obspy
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib notebook
import locator as loc

ice_thickness = 3450
shear_wave_speed = 2000
trial_loc = 1000
traces = []

start_time = time.perf_counter()
file_list = ('TIME_WAIS_11_20190105_20190116.mseed','TIME_WAIS_500_20190105_20190116.mseed', \
             'TIME_WAIS_740_20190105_20190116.mseed', 'TIME_WAIS_1010_20190105_20190116.mseed', \
             'TIME_WAIS_1520_20190105_20190116.mseed', 'TIME_WAIS_2000_20190105_20190116.mseed', \
             'TIME_WAIS_2510_20190105_20190116.mseed')

station_location_x = np.array([11, 500, 740, 1010, 1520, 2000, 2510])
index_list = (24,25,24,25,25,22,22)

for file_name, i in zip(file_list, index_list):
    
    st=obspy.read(file_name)
    st.filter('highpass', freq = 10.0)
    tr = st[i] 
    traces.append(tr)
    del(st)
    del(tr)
    print('Loaded file in %f seconds'%(time.perf_counter()-start_time))

    #Below runs code with synthetic data
#noise = np.random.uniform(low=1, high=, size=(1000,))
#t = np.arange(0, 1, 0.001)
#trace = np.exp(-(t-0.5)**2/0.001)
#station_location_x = np.array([0,10, 35, 400, 675, 890, 1000, 1260, 1780, 1920])
#for x in (station_location_x):
#    offset = loc.tt(ice_thickness, x, station_location_x[0], trial_loc, shear_wave_speed)
#    traces.append(loc.fft_shift(t,trace,offset))
    
#fig = plt.subplots()    
#for tr in (traces):
#    plt.plot(t, np.real(tr))
#plt.show()

Loaded file in 35.319481 seconds
Loaded file in 78.062432 seconds
Loaded file in 115.441832 seconds
Loaded file in 152.807750 seconds
Loaded file in 188.778244 seconds
Loaded file in 222.923543 seconds
Loaded file in 257.276294 seconds


In [2]:
#Test Shift Function

t = np.arange(0, 1, 0.001)
trace = np.exp(-(t-0.5)**2/0.001)
offset = 0.1
shifted_trace = loc.fft_shift(t,trace,offset)
plt.plot(t, trace)
plt.plot(t, np.real(shifted_trace))
plt.show()

<IPython.core.display.Javascript object>

In [8]:
xm = np.mean(station_location_x)
locations = np.arange(xm-ice_thickness,xm+ice_thickness,10)
relative_travel_time = np.zeros(locations.shape)
best = np.zeros(locations.shape)
t = traces[0].times()
#subset_index = len(t)
subset_index = 1080
num_iterations = int(np.floor(len(t) / subset_index))

start_time = time.perf_counter()
fig_num = 0
w = 25

def backproject(locations, traces, subset_index, chunk_index, w, ice_thickness, station_location_x, shear_wave_speed):
    big_matrix = np.zeros((subset_index-w, len(locations)))
    for i in np.arange(0,len(locations)):
        shifted_traces = []
        for j in range(0, len(traces)):
            current_trace = traces[j][((subset_index * chunk_index) + 1): subset_index * (chunk_index + 1)]
            tr_avg = (np.convolve(np.abs(current_trace), np.ones(w), 'valid') / w)
#             tr_norm = current_trace[(w-1):] / tr_avg[0:]
            tr_norm = tr_avg[0:] / np.mean(tr_avg[0:])
#             tr_norm = current_trace  # for synthetic data + noise
            shifted_traces.append(loc.fft_shift(t[((subset_index * chunk_index) + 1): subset_index * (chunk_index + 1)],\
                                            tr_norm, loc.tt(ice_thickness, station_location_x[j], station_location_x[0], \
                                                        locations[i], shear_wave_speed)))
            #print('          Finished trace loop %d in %f seconds'%(j,time.perf_counter()-start_time))
        stack = sum(shifted_traces)
        big_matrix[:,i] = np.real(stack)
        #print('     Finished location loop %d in %f seconds'%(i,time.perf_counter()-start_time))
    return big_matrix
    
# fig = plt.subplots()
gs_kw = dict(width_ratios=[1,2], height_ratios=[1,2])
fig = plt.subplots(2,2,sharex=True,gridspec_kw=gs_kw)
for chunk_index in range(0,num_iterations+1):
    big_matrix = backproject(locations, traces, subset_index, chunk_index, w, ice_thickness, station_location_x, shear_wave_speed)

    plt.subplot(2,2,2)
    hi_val = np.percentile(np.abs(big_matrix), 95)
    lo_val = np.percentile(np.abs(big_matrix), 25)
    plt.rcParams['pcolor.shading'] = 'auto'
    c = plt.pcolormesh(locations, t[((subset_index * chunk_index) + 1):subset_index * (chunk_index + 1) - w + 1],\
                               np.abs(big_matrix), vmax = hi_val, vmin=lo_val)
#     plt.axis('scaled')
#     plt.colorbar(c)
    plt.subplot(2,2,4)
    plt.plot(locations,np.max(np.abs(big_matrix),axis=0))
    
    plt.subplot(2,2,1)
    plt.plot(np.max(np.abs(big_matrix),axis=1),t[((subset_index * chunk_index) + 1):subset_index * (chunk_index + 1) - w + 1])
    
#     plt.show()
    
    fig_num += 1
    plt.savefig('Smoothed_Figure_%d'%fig_num)
    plt.clf()
    
    
    
    del(big_matrix)
    print('Finished time loop %d in %f seconds'%(chunk_index,time.perf_counter()-start_time))

<IPython.core.display.Javascript object>

Finished time loop 0 in 3.129590 seconds
Finished time loop 1 in 6.190039 seconds
Finished time loop 2 in 9.244978 seconds
Finished time loop 3 in 12.383453 seconds
Finished time loop 4 in 15.420257 seconds
Finished time loop 5 in 18.467158 seconds
Finished time loop 6 in 21.506923 seconds
Finished time loop 7 in 24.522665 seconds
Finished time loop 8 in 27.584953 seconds
Finished time loop 9 in 30.638861 seconds
Finished time loop 10 in 33.688304 seconds
Finished time loop 11 in 36.743046 seconds
Finished time loop 12 in 39.810058 seconds
Finished time loop 13 in 42.947743 seconds
Finished time loop 14 in 45.986871 seconds
Finished time loop 15 in 49.021960 seconds
Finished time loop 16 in 52.058414 seconds
Finished time loop 17 in 55.119050 seconds
Finished time loop 18 in 58.169634 seconds
Finished time loop 19 in 61.221857 seconds
Finished time loop 20 in 64.300403 seconds
Finished time loop 21 in 67.359457 seconds
Finished time loop 22 in 70.489448 seconds
Finished time loop 23 in

Finished time loop 186 in 571.078860 seconds
Finished time loop 187 in 574.108338 seconds
Finished time loop 188 in 577.145254 seconds
Finished time loop 189 in 580.209518 seconds
Finished time loop 190 in 583.250624 seconds
Finished time loop 191 in 586.301499 seconds
Finished time loop 192 in 589.365533 seconds
Finished time loop 193 in 592.410252 seconds
Finished time loop 194 in 595.457737 seconds
Finished time loop 195 in 598.589660 seconds
Finished time loop 196 in 601.630080 seconds
Finished time loop 197 in 604.684858 seconds
Finished time loop 198 in 607.735209 seconds
Finished time loop 199 in 610.787644 seconds
Finished time loop 200 in 613.844897 seconds
Finished time loop 201 in 616.900459 seconds
Finished time loop 202 in 619.957630 seconds
Finished time loop 203 in 623.011529 seconds
Finished time loop 204 in 626.062257 seconds
Finished time loop 205 in 629.193953 seconds
Finished time loop 206 in 632.222259 seconds
Finished time loop 207 in 635.266133 seconds
Finished t

Finished time loop 368 in 1127.627267 seconds
Finished time loop 369 in 1130.677830 seconds
Finished time loop 370 in 1133.724807 seconds
Finished time loop 371 in 1136.759638 seconds
Finished time loop 372 in 1139.821458 seconds
Finished time loop 373 in 1142.880008 seconds
Finished time loop 374 in 1145.941862 seconds
Finished time loop 375 in 1148.991482 seconds
Finished time loop 376 in 1152.056751 seconds
Finished time loop 377 in 1155.128116 seconds
Finished time loop 378 in 1158.269149 seconds
Finished time loop 379 in 1161.324061 seconds
Finished time loop 380 in 1164.361983 seconds
Finished time loop 381 in 1167.406231 seconds
Finished time loop 382 in 1170.468323 seconds
Finished time loop 383 in 1173.511524 seconds
Finished time loop 384 in 1176.563933 seconds
Finished time loop 385 in 1179.640731 seconds
Finished time loop 386 in 1182.694389 seconds
Finished time loop 387 in 1185.844721 seconds
Finished time loop 388 in 1188.897476 seconds
Finished time loop 389 in 1191.949

Finished time loop 547 in 1675.183130 seconds
Finished time loop 548 in 1678.193676 seconds
Finished time loop 549 in 1681.225029 seconds
Finished time loop 550 in 1684.285018 seconds
Finished time loop 551 in 1687.400571 seconds
Finished time loop 552 in 1690.445824 seconds
Finished time loop 553 in 1693.479016 seconds
Finished time loop 554 in 1696.512497 seconds
Finished time loop 555 in 1699.559214 seconds
Finished time loop 556 in 1702.608721 seconds
Finished time loop 557 in 1705.656958 seconds
Finished time loop 558 in 1708.702806 seconds
Finished time loop 559 in 1711.742996 seconds
Finished time loop 560 in 1714.790157 seconds
Finished time loop 561 in 1717.919695 seconds
Finished time loop 562 in 1720.939770 seconds
Finished time loop 563 in 1723.967599 seconds
Finished time loop 564 in 1727.006297 seconds
Finished time loop 565 in 1730.047540 seconds
Finished time loop 566 in 1733.097214 seconds
Finished time loop 567 in 1736.138711 seconds
Finished time loop 568 in 1739.182

...

Finished time loop 1223 in 3744.425428 seconds
Finished time loop 1224 in 3747.489904 seconds
Finished time loop 1225 in 3750.551109 seconds
Finished time loop 1226 in 3753.602741 seconds
Finished time loop 1227 in 3756.663739 seconds
Finished time loop 1228 in 3759.741240 seconds
Finished time loop 1229 in 3762.819805 seconds
Finished time loop 1230 in 3765.961333 seconds
Finished time loop 1231 in 3768.985461 seconds
Finished time loop 1232 in 3772.024498 seconds
Finished time loop 1233 in 3775.072192 seconds
Finished time loop 1234 in 3778.121658 seconds
Finished time loop 1235 in 3781.176431 seconds
Finished time loop 1236 in 3784.193954 seconds
Finished time loop 1237 in 3787.272786 seconds
Finished time loop 1238 in 3790.346910 seconds
Finished time loop 1239 in 3793.409614 seconds
Finished time loop 1240 in 3796.554380 seconds
Finished time loop 1241 in 3799.591840 seconds
Finished time loop 1242 in 3802.640032 seconds
Finished time loop 1243 in 3805.684395 seconds
Finished time

Finished time loop 1398 in 4279.914839 seconds
Finished time loop 1399 in 4282.976823 seconds
Finished time loop 1400 in 4286.032013 seconds
Finished time loop 1401 in 4289.087742 seconds
Finished time loop 1402 in 4292.220078 seconds
Finished time loop 1403 in 4295.274086 seconds
Finished time loop 1404 in 4298.335317 seconds
Finished time loop 1405 in 4301.343305 seconds
Finished time loop 1406 in 4304.398851 seconds
Finished time loop 1407 in 4307.450283 seconds
Finished time loop 1408 in 4310.510164 seconds
Finished time loop 1409 in 4313.564288 seconds
Finished time loop 1410 in 4316.612686 seconds
Finished time loop 1411 in 4319.674372 seconds
Finished time loop 1412 in 4322.794908 seconds
Finished time loop 1413 in 4325.865682 seconds
Finished time loop 1414 in 4328.924109 seconds
Finished time loop 1415 in 4331.981295 seconds
Finished time loop 1416 in 4335.044547 seconds
Finished time loop 1417 in 4338.113686 seconds
Finished time loop 1418 in 4341.193022 seconds
Finished time

Finished time loop 1573 in 4811.766970 seconds
Finished time loop 1574 in 4814.733838 seconds
Finished time loop 1575 in 4817.710493 seconds
Finished time loop 1576 in 4820.682674 seconds
Finished time loop 1577 in 4823.739389 seconds
Finished time loop 1578 in 4826.702050 seconds
Finished time loop 1579 in 4829.670710 seconds
Finished time loop 1580 in 4832.630924 seconds
Finished time loop 1581 in 4835.601175 seconds
Finished time loop 1582 in 4838.613003 seconds
Finished time loop 1583 in 4841.585960 seconds
Finished time loop 1584 in 4844.574304 seconds
Finished time loop 1585 in 4847.574265 seconds
Finished time loop 1586 in 4850.566540 seconds
Finished time loop 1587 in 4853.675109 seconds
Finished time loop 1588 in 4856.661021 seconds
Finished time loop 1589 in 4859.643044 seconds
Finished time loop 1590 in 4862.623236 seconds
Finished time loop 1591 in 4865.603936 seconds
Finished time loop 1592 in 4868.593492 seconds
Finished time loop 1593 in 4871.588944 seconds
Finished time

Finished time loop 1748 in 5337.118446 seconds
Finished time loop 1749 in 5340.197298 seconds
Finished time loop 1750 in 5343.269728 seconds
Finished time loop 1751 in 5346.432969 seconds
Finished time loop 1752 in 5349.490291 seconds
Finished time loop 1753 in 5352.540893 seconds
Finished time loop 1754 in 5355.595577 seconds
Finished time loop 1755 in 5358.656931 seconds
Finished time loop 1756 in 5361.738233 seconds
Finished time loop 1757 in 5364.825514 seconds
Finished time loop 1758 in 5367.899337 seconds
Finished time loop 1759 in 5370.965658 seconds
Finished time loop 1760 in 5374.033556 seconds
Finished time loop 1761 in 5377.173952 seconds
Finished time loop 1762 in 5380.234776 seconds
Finished time loop 1763 in 5383.286933 seconds
Finished time loop 1764 in 5386.345843 seconds
Finished time loop 1765 in 5389.422381 seconds
Finished time loop 1766 in 5392.480596 seconds
Finished time loop 1767 in 5395.553206 seconds
Finished time loop 1768 in 5398.619147 seconds
Finished time

KeyboardInterrupt: 